<a href="https://colab.research.google.com/github/guilhermegandra/segmentador_PhD/blob/master/COMPARATIVO_EDIFICA%C3%87%C3%95ES_VALE_DO_TAQUARI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

# Caminho para a pasta TESE
tese_folder = "/content/drive/My Drive/TESE"

# Listar arquivos na pasta
print("Arquivos na pasta TESE:")
print(os.listdir(tese_folder))


Arquivos na pasta TESE:
['REFERENCIAL.prj', 'REFERENCIAL.shp', 'REFERENCIAL.shx', 'REFERENCIAL.cpg', 'REFERENCIAL.dbf', 'REFERENCIAL.shp.xml', 'REFERENCIAL.sbn', 'REFERENCIAL.sbx', 'OSM.prj', 'OSM.shx', 'OSM.shp', 'OSM.cpg', 'OSM.dbf', 'OSM.shp.xml', 'OSM.sbn', 'OSM.sbx', 'GOOGLE.prj', 'GOOGLE.shx', 'GOOGLE.shp', 'GOOGLE.cpg', 'GOOGLE.dbf', 'GOOGLE.shp.xml', 'GOOGLE.sbn', 'GOOGLE.sbx', 'REFERENCIAL.shp.DESKTOP-DOCPV23.17840.32712.sr.lock', 'OSM.shp.DESKTOP-DOCPV23.17840.32712.sr.lock', 'GOOGLE.shp.DESKTOP-DOCPV23.17840.32712.sr.lock']


In [ ]:
import geopandas as gpd

# Caminho dos arquivos na pasta TESE
referencial_path = os.path.join(tese_folder, "REFERENCIAL.shp")
osm_path = os.path.join(tese_folder, "OSM.shp")
google_path = os.path.join(tese_folder, "GOOGLE.shp")

# Carregar os shapefiles
referencial = gpd.read_file(referencial_path)
osm = gpd.read_file(osm_path)
google = gpd.read_file(google_path)

# Verificar a carga
print(f"Feições no REFERENCIAL: {len(referencial)}")
print(f"Feições no OSM: {len(osm)}")
print(f"Feições no GOOGLE: {len(google)}")


Feições no REFERENCIAL: 4199
Feições no OSM: 3930
Feições no GOOGLE: 6122


In [ ]:
# Reprojetar para CRS métrico pois estão em WGS
referencial = referencial.to_crs(epsg=3857)
osm = osm.to_crs(epsg=3857)
google = google.to_crs(epsg=3857)


In [ ]:
def jaccard_index(polygon1, polygon2):
    """
    Calcula o índice de Jaccard entre dois polígonos.
    """
    intersection = polygon1.intersection(polygon2).area
    union = polygon1.union(polygon2).area
    return intersection / union if union > 0 else 0


In [ ]:
# Reprojetar para CRS métrico
referencial = referencial.to_crs(epsg=3857)
osm = osm.to_crs(epsg=3857)
google = google.to_crs(epsg=3857)

# Função Jaccard Index
def jaccard_index(polygon1, polygon2):
    intersection = polygon1.intersection(polygon2).area
    union = polygon1.union(polygon2).area
    return intersection / union if union > 0 else 0

# Comparação entre REFERENCIAL e os outros conjuntos
results = []

for idx, ref_geom in enumerate(referencial.geometry):
    # Encontra a geometria correspondente no OSM e GOOGLE mais próxima
    osm_match = osm.iloc[osm.distance(ref_geom).idxmin()]
    google_match = google.iloc[google.distance(ref_geom).idxmin()]

    # Calcula índices de Jaccard
    osm_jaccard = jaccard_index(ref_geom, osm_match.geometry)
    google_jaccard = jaccard_index(ref_geom, google_match.geometry)

    # Registra os resultados
    results.append({
        "Referencial_ID": idx,
        "OSM_Jaccard": osm_jaccard,
        "Google_Jaccard": google_jaccard,
        "Better_Match": "OSM" if osm_jaccard > google_jaccard else "GOOGLE"
    })

# Criar DataFrame com os resultados
import pandas as pd
results_df = pd.DataFrame(results)

# Estatísticas gerais
print("Média Jaccard - OSM:", results_df["OSM_Jaccard"].mean())
print("Média Jaccard - Google:", results_df["Google_Jaccard"].mean())
print("Melhor conjunto em geral:", results_df["Better_Match"].value_counts().idxmax())


Média Jaccard - OSM: 0.31893489540776465
Média Jaccard - Google: 0.34474666439081736
Melhor conjunto em geral: GOOGLE


In [ ]:
import os

# Listar todos os arquivos no diretório atual
print("Arquivos no diretório atual:")
print(os.listdir())


Arquivos no diretório atual:
['.config', 'drive', 'sample_data']


In [ ]:
# Certifique-se de que este código está sendo executado corretamente
results_df.to_csv("comparacao_jaccard.csv", index=False)


In [ ]:
from google.colab import files

# Tente novamente baixar o arquivo
files.download("comparacao_jaccard.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Criar GeoDataFrame para os resultados
from geopandas import GeoDataFrame

geometry = []
for idx, ref_geom in enumerate(referencial.geometry):
    osm_match = osm.iloc[osm.distance(ref_geom).idxmin()]
    google_match = google.iloc[google.distance(ref_geom).idxmin()]

    # Salvar apenas a geometria correspondente (com maior Jaccard)
    if results[idx]["Better_Match"] == "OSM":
        geometry.append(osm_match.geometry)
    else:
        geometry.append(google_match.geometry)

# Criar GeoDataFrame final
results_gdf = GeoDataFrame(results_df, geometry=geometry, crs=referencial.crs)


In [ ]:
# Soma dos Jaccards e número de correspondências
osm_jaccard_sum = results_df["OSM_Jaccard"].sum()
google_jaccard_sum = results_df["Google_Jaccard"].sum()

# Número de correspondências efetivas (índices > 0)
osm_matches = (results_df["OSM_Jaccard"] > 0).sum()
google_matches = (results_df["Google_Jaccard"] > 0).sum()

# Jaccard Normalizado
osm_normalized = osm_jaccard_sum / osm_matches
google_normalized = google_jaccard_sum / google_matches

print(f"Jaccard Normalizado - OSM: {osm_normalized}")
print(f"Jaccard Normalizado - Google: {google_normalized}")


Jaccard Normalizado - OSM: 0.35990530121397574
Jaccard Normalizado - Google: 0.3639907578016198
